In [4]:
import torch as t
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision as tv
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

from multiprocessing import cpu_count

device = t.device('cuda') if t.cuda.is_available() else t.device('cpu')

In [5]:
transform = transforms.ToTensor()

train_dataset = tv.datasets.CIFAR10(root='~/cifar',train=True,download=True,transform=transform)
test_dataset = tv.datasets.CIFAR10(root='~/cifar',train=False,download=False,transform=transform)

train_loader = DataLoader(train_dataset, shuffle=True, batch_size=128, num_workers=cpu_count())
test_loader = DataLoader(test_dataset, shuffle=True, batch_size=128, num_workers=cpu_count())

Extracting C:\Users\itomm/cifar\cifar-10-python.tar.gz to C:\Users\itomm/cifar


In [6]:
class ImageShape(NamedTuple):
    h: int
    w: int
    c: int

class Net(nn.Module):
    def __init__(
        self,
        height: int = 32,
        width: int = 32,
        channels: int = 3,
        class_count: int = 10
    ):
        self.input_shape = ImageShape(height, width, channels)
        self.class_count = class_count
        
#         self.conv1 = nn.Conv2d(
#             in_channels=self.input_shape.c,
#             out_channels=32,
#             kernel_size=(5,5),
#             padding=(3,3)
#         )
        """
        nn.Conv2d(
            in_channels: int,
            out_channels: int,
            kernel_size: int, tuple
                an int uses the same W x H dimensions
                a tuple uses (W, H) as dimensions
            stride: int, tuple
            padding: int, tuple
        )
        """
        self.conv1 = nn.Conv2d(self.input_shape.c, 32, 5, padding=3)
        
        # self.pool = nn.MaxPool2d(kernel_size=(2, 2), stride=(2,2))
        """
        nn.MaxPool2d(
            kernel_size: int, tuple
                an int uses the same W x H dimensions
                a tuple uses (W, H) as dimensions
            stride: int, tuple
                (same as above)
        )
        """
        self.pool1 = nn.MaxPool2d(2,2)
        
        self.conv2 = nn.Conv2d(32, 64, 5, padding=3)
        self.pool2 = nn.MaxPool2d(8, 8)
        self.fc1 = nn.Linear(4096, 1024)
        self.fc2 = nn.Linear(1024, 10)
        self.initialise_layer(self.conv1)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    @staticmethod
    def initialise_layer(layer):
        if hasattr(layer, "bias"):
            nn.init.zeros_(layer.bias)
        if hasattr(layer, "weight"):
            nn.init.kaiming_normal_(layer.weight)

In [22]:
xd = t.randn(1,3,32,32)
print(xd.shape)
conv1 = nn.Conv2d(3, 32, 5, padding=2)
xd = F.relu(conv1(xd))
print(xd.shape)
pool = nn.MaxPool2d(2,2)
xd = pool(xd)
print(xd.shape)
conv2 = nn.Conv2d(32,64,5, padding=2)
xd = F.relu(conv2(xd))
print(xd.shape)
xd = pool(xd)
print(xd.shape)
xd = t.flatten(xd, 1)
print(xd.shape)
fc1 = nn.Linear(4096, 1024)
fc2 = nn.Linear(1024, 10)
xd = F.relu(fc1(xd))
print(xd.shape)
xd = fc2(xd)
print(xd.shape)
print(xd)

torch.Size([1, 3, 32, 32])
torch.Size([1, 32, 32, 32])
torch.Size([1, 32, 16, 16])
torch.Size([1, 64, 16, 16])
torch.Size([1, 64, 8, 8])
torch.Size([1, 4096])
torch.Size([1, 1024])
torch.Size([1, 10])
tensor([[-0.2317, -0.0547,  0.0782,  0.0770, -0.1781, -0.0291,  0.0751,  0.0072,
          0.1072, -0.0551]], grad_fn=<AddmmBackward>)
